In [1]:
✅ Final Suggested Schema for Your Dataset


Column	Type	Required	Purpose
latitude	float	✅ Yes	Needed for spatial interpolation
longitude	float	✅ Yes	Needed for spatial interpolation
timestamp	datetime	✅ Yes	For extracting temporal patterns
co2	float	✅ Yes	Pollution indicator (can be your main target)
pm2_5	float	✅ Yes	Fine particles, another pollution target
pm10	float	✅ Yes	Coarser particles, also important
humidity	float	Optional	Affects particle suspension and dispersion
temperature	float	Optional	Influences chemical reactions and pollutant behavior
wind_speed	float	Optional	Strong wind → disperses pollutants
pressure	float	Optional	Can impact air quality readings
traffic_index	float/int	Optional	Strong signal for urban pollution levels

SyntaxError: invalid character '✅' (U+2705) (552190771.py, line 1)

In [2]:
latitude | longitude | timestamp | co2 | pm2_5 | pm10 | humidity | temperature | wind_speed | pressure | traffic_index
44.4328 | 26.1043 | 2023-04-12 09:15:00 | 410 | 12.3 | 25.6 | 55 | 21.0°C | 2.4 | 1012 | 73

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (3733562308.py, line 2)

In [24]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os

API_KEY = "41a32c6d87e64eb16c24f687a699b887a98aad1786c3f6f0376aad1da0fdca86"  # 🔐 Paste your OpenAQ API key here

HEADERS = {
    "X-API-Key": API_KEY
}

def get_all_locations(limit=500):
    url = "https://api.openaq.org/v3/locations"
    all_locations = []
    page = 1

    while True:
        print(f"🌍 Fetching page {page}...")
        response = requests.get(url, headers=HEADERS, params={
            "limit": limit,
            "page": page,
            "sort": "desc"
        })

        results = response.json().get("results", [])
        if not results:
            break

        all_locations.extend(results)
        page += 1

        if page > 4:  # Avoid downloading thousands of entries
            break

    print(f"✅ Retrieved {len(all_locations)} total locations.")
    return all_locations

def filter_bucharest_stations(locations):
    return [
        loc for loc in locations
        if (
            loc.get("coordinates") and
            abs(loc["coordinates"]["latitude"] - 44.43) < 0.3 and
            abs(loc["coordinates"]["longitude"] - 26.10) < 0.3
        )
    ]

def fetch_measurements(location_name, parameters=["pm25", "pm10", "co"], days_back=7):
    measurements = []
    base_url = "https://api.openaq.org/v3/measurements"
    date_to = datetime.utcnow()
    date_from = date_to - timedelta(days=days_back)

    for param in parameters:
        page = 1
        while True:
            response = requests.get(base_url, headers=HEADERS, params={
                "location": location_name,
                "parameter": param,
                "date_from": date_from.isoformat() + "Z",
                "date_to": date_to.isoformat() + "Z",
                "limit": 100,
                "page": page,
                "sort": "desc"
            })

            data = response.json().get("results", [])
            if not data:
                break

            for entry in data:
                measurements.append({
                    "timestamp": entry["date"]["utc"],
                    "location": entry["location"],
                    "parameter": entry["parameter"],
                    "value": entry["value"],
                    "unit": entry["unit"],
                    "latitude": entry["coordinates"]["latitude"],
                    "longitude": entry["coordinates"]["longitude"]
                })

            page += 1

    return measurements

# 🚀 Run the flow
print("🔍 Fetching global locations...")
locations = get_all_locations()

print("📍 Filtering by Bucharest coordinates...")
bucharest_stations = filter_bucharest_stations(locations)

if not bucharest_stations:
    print("❌ No stations found near Bucharest.")
else:
    print(f"✅ Found {len(bucharest_stations)} Bucharest station(s):")
    for s in bucharest_stations:
        print(" -", s["name"])

    all_data = []
    for station in bucharest_stations:
        print(f"📡 Downloading from {station['name']}...")
        data = fetch_measurements(station["name"])
        all_data.extend(data)

    if all_data:
        df = pd.DataFrame(all_data)
        os.makedirs("data", exist_ok=True)
        df.to_csv("data/bucharest_openaq_data.csv", index=False)
        print("✅ Saved to data/bucharest_openaq_data.csv")
        display(df.head())
    else:
        print("⚠️ No measurement data found.")


🔍 Fetching global locations...
🌍 Fetching page 1...
🌍 Fetching page 2...
🌍 Fetching page 3...
🌍 Fetching page 4...
✅ Retrieved 2000 total locations.
📍 Filtering by Bucharest coordinates...
❌ No stations found near Bucharest.


In [ ]:
### OpenAQ does not have a valid station for Bucharest 

In [ ]:
### Creating dataset: https://www.eea.europa.eu/en/analysis/maps-and-charts/index?activeTab=265e2bee-7de3-46e8-b6ee-76005f3f434f - for pm10

### PM10

In [61]:
import pandas as pd
from pathlib import Path

# Folder where your files are
folder = Path("data/pm10-2022-e1a")

# Match all files named file1.parquet to file6.parquet
files = sorted(folder.glob("SPO-*.parquet"))

# Read and concatenate them
dfs = [pd.read_parquet(f) for f in files]
df_all = pd.concat(dfs, ignore_index=True)

# Extract station code from Samplingpoint
df_all["station_code"] = df_all["Samplingpoint"].str.extract(r"(RO\d{4}A)")

# Filter only for RO0070A (B-6)
df_b4 = df_all[df_all["station_code"] == "RO0065A"].copy()

print(f"✅ Found {len(df_b4)} rows for station B-4 (RO0065A)")
df_b4.head()


✅ Found 8760 rows for station B-4 (RO0065A)


,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog,station_code
0,RO/SPO-RO0065A_00005_101,5,2021-12-31 23:00:00,2022-01-01 00:00:00,0E-18,ug.m-3,hour,-1,1,2023-09-26 14:37:36,None,9257d02c-fa75-4767-bed6-ee7d1f4a2f3a,RO0065A
1,RO/SPO-RO0065A_00005_101,5,2022-01-01 00:00:00,2022-01-01 01:00:00,0E-18,ug.m-3,hour,-1,1,2023-09-26 14:37:36,None,9257d02c-fa75-4767-bed6-ee7d1f4a2f3a,RO0065A
2,RO/SPO-RO0065A_00005_101,5,2022-01-01 01:00:00,2022-01-01 02:00:00,0E-18,ug.m-3,hour,-1,1,2023-09-26 14:37:36,None,9257d02c-fa75-4767-bed6-ee7d1f4a2f3a,RO0065A
3,RO/SPO-RO0065A_00005_101,5,2022-01-01 02:00:00,2022-01-01 03:00:00,0E-18,ug.m-3,hour,-1,1,2023-09-26 14:37:36,None,9257d02c-fa75-4767-bed6-ee7d1f4a2f3a,RO0065A
4,RO/SPO-RO0065A_00005_101,5,2022-01-01 03:00:00,2022-01-01 04:00:00,0E-18,ug.m-3,hour,-1,1,2023-09-26 14:37:36,None,9257d02c-fa75-4767-bed6-ee7d1f4a2f3a,RO0065A


In [63]:
# 1. Selectăm doar coloanele necesare
df_pm10 = df_b4[["Start", "End", "Value", "Unit"]].copy()

# 2. Renumim coloanele
df_pm10.rename(columns={
    "Start": "start",
    "End": "end",
    "Value": "pm10",
    "Unit": "pm10_unit"
}, inplace=True)

# 3. Filtrăm valorile valide
df_pm10_filtered = df_pm10[df_pm10["pm10"] > 1e-6].copy()
df_pm10_filtered.reset_index(drop=True, inplace=True)

# 4. Adăugăm coordonatele stației B-4 (RO0065A)
df_pm10_filtered["latitude"] = 44.4264
df_pm10_filtered["longitude"] = 26.1406

# 5. Salvăm dataset-ul
df_pm10_filtered.to_csv("data/pm10-2022-e1a/pm10_bucharest_clean-2022.csv", index=False)

print(f"✅ Cleaned dataset saved. Remaining rows: {len(df_pm10_filtered)}")
df_pm10_filtered.head()



✅ Cleaned dataset saved. Remaining rows: 8442


,start,end,pm10,pm10_unit,latitude,longitude
0,2022-01-01 23:00:00,2022-01-02 00:00:00,46.390000000000000000,ug.m-3,44.4264,26.1406
1,2022-01-02 00:00:00,2022-01-02 01:00:00,46.480000000000000000,ug.m-3,44.4264,26.1406
2,2022-01-02 01:00:00,2022-01-02 02:00:00,33.500000000000000000,ug.m-3,44.4264,26.1406
3,2022-01-02 02:00:00,2022-01-02 03:00:00,19.340000000000000000,ug.m-3,44.4264,26.1406
4,2022-01-02 03:00:00,2022-01-02 04:00:00,14.990000000000000000,ug.m-3,44.4264,26.1406


### CO2

In [78]:
# import pandas as pd
# from pathlib import Path

# # Folder where your files are
# folder = Path("data/co2-2022-e1a")

# # Match all files named file1.parquet to file6.parquet
# files = sorted(folder.glob("SPO-*.parquet"))

# # Read and concatenate them
# dfs = [pd.read_parquet(f) for f in files]
# df_all = pd.concat(dfs, ignore_index=True)

# # Preview
# print(f"✅ Loaded {len(df_all)} rows from {len(files)} files")
# df_all.head(-1)

### PM2.5

In [67]:
import pandas as pd
from pathlib import Path

# Folder where your files are
folder = Path("data/pm2.5-2022-e1a")

# Match all files named file1.parquet to file6.parquet
files = sorted(folder.glob("SPO-*.parquet"))

# Read and concatenate them
dfs = [pd.read_parquet(f) for f in files]
df_all = pd.concat(dfs, ignore_index=True)

# Extract station code from Samplingpoint
df_all["station_code"] = df_all["Samplingpoint"].str.extract(r"(RO\d{4}A)")

# Filter only for RO0070A (B-6)
df_b4 = df_all[df_all["station_code"] == "RO0065A"].copy()

print(f"✅ Found {len(df_b4)} rows for station B-4 (RO0065A)")
df_b4.head()

✅ Found 8759 rows for station B-4 (RO0065A)


,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog,station_code
0,RO/SPO-RO0065A_06001_101,6001,2022-01-01 00:00:00,2022-01-01 01:00:00,97.230000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,3443a30d-a26d-4740-ad8a-1212ddbf7a7e,RO0065A
1,RO/SPO-RO0065A_06001_101,6001,2022-01-01 01:00:00,2022-01-01 02:00:00,58.210000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,3443a30d-a26d-4740-ad8a-1212ddbf7a7e,RO0065A
2,RO/SPO-RO0065A_06001_101,6001,2022-01-01 02:00:00,2022-01-01 03:00:00,41.240000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,3443a30d-a26d-4740-ad8a-1212ddbf7a7e,RO0065A
3,RO/SPO-RO0065A_06001_101,6001,2022-01-01 03:00:00,2022-01-01 04:00:00,31.920000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,3443a30d-a26d-4740-ad8a-1212ddbf7a7e,RO0065A
4,RO/SPO-RO0065A_06001_101,6001,2022-01-01 04:00:00,2022-01-01 05:00:00,29.190000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,3443a30d-a26d-4740-ad8a-1212ddbf7a7e,RO0065A


In [77]:
# 1. Selectăm doar coloanele necesare
df_pm25 = df_b4[["Start", "End", "Value", "Unit"]].copy()

# 2. Renumim coloanele
df_pm25.rename(columns={
    "Start": "start",
    "End": "end",
    "Value": "pm2_5",
    "Unit": "pm2_5_unit"
}, inplace=True)

# 3. Filtrăm valorile valide
df_pm25_filtered = df_pm25[df_pm25["pm2_5"] > 1e-6].copy()
df_pm25_filtered.reset_index(drop=True, inplace=True)

# 4. Adăugăm coordonatele stației B-4 (RO0065A)
df_pm25_filtered["latitude"] = 44.4264
df_pm25_filtered["longitude"] = 26.1406

# 5. Salvăm dataset-ul
df_pm25_filtered.to_csv("data/pm2.5-2022-e1a/pm25_bucharest_clean-2022.csv", index=False)

print(f"✅ Cleaned dataset saved. Remaining rows: {len(df_pm25_filtered)}")
df_pm25_filtered.head()



✅ Cleaned dataset saved. Remaining rows: 8441


,start,end,pm2_5,pm2_5_unit,latitude,longitude
0,2022-01-01 00:00:00,2022-01-01 01:00:00,97.230000000000000000,ug.m-3,44.4264,26.1406
1,2022-01-01 01:00:00,2022-01-01 02:00:00,58.210000000000000000,ug.m-3,44.4264,26.1406
2,2022-01-01 02:00:00,2022-01-01 03:00:00,41.240000000000000000,ug.m-3,44.4264,26.1406
3,2022-01-01 03:00:00,2022-01-01 04:00:00,31.920000000000000000,ug.m-3,44.4264,26.1406
4,2022-01-01 04:00:00,2022-01-01 05:00:00,29.190000000000000000,ug.m-3,44.4264,26.1406


In [38]:
### NO2

In [82]:
import pandas as pd
from pathlib import Path

# Folder where your files are
folder = Path("data/no2-2022-e1a")

# Match all files named file1.parquet to file6.parquet
files = sorted(folder.glob("SPO-*.parquet"))

# Read and concatenate them
dfs = [pd.read_parquet(f) for f in files]
df_all = pd.concat(dfs, ignore_index=True)

# Extract station code from Samplingpoint
df_all["station_code"] = df_all["Samplingpoint"].str.extract(r"(RO\d{4}A)")

# Filter only for RO0070A (B-6)
df_b4 = df_all[df_all["station_code"] == "RO0065A"].copy()

print(f"✅ Found {len(df_b4)} rows for station B-4 (RO0065A)")
df_b4.head()

✅ Found 8759 rows for station B-4 (RO0065A)


,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog,station_code
0,RO/SPO-RO0065A_00008_100,8,2022-01-01 00:00:00,2022-01-01 01:00:00,17.870000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,e719fc6b-a4b3-48dd-9d73-3104e6eb29ec,RO0065A
1,RO/SPO-RO0065A_00008_100,8,2022-01-01 01:00:00,2022-01-01 02:00:00,18.270000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,e719fc6b-a4b3-48dd-9d73-3104e6eb29ec,RO0065A
2,RO/SPO-RO0065A_00008_100,8,2022-01-01 02:00:00,2022-01-01 03:00:00,13.840000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,e719fc6b-a4b3-48dd-9d73-3104e6eb29ec,RO0065A
3,RO/SPO-RO0065A_00008_100,8,2022-01-01 03:00:00,2022-01-01 04:00:00,15.650000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,e719fc6b-a4b3-48dd-9d73-3104e6eb29ec,RO0065A
4,RO/SPO-RO0065A_00008_100,8,2022-01-01 04:00:00,2022-01-01 05:00:00,18.380000000000000000,ug.m-3,hour,1,1,2023-09-26 14:37:36,None,e719fc6b-a4b3-48dd-9d73-3104e6eb29ec,RO0065A


In [83]:
# 1. Selectăm doar coloanele necesare
df_no2 = df_b4[["Start", "End", "Value", "Unit"]].copy()

# 2. Renumim coloanele
df_no2.rename(columns={
    "Start": "start",
    "End": "end",
    "Value": "no2",
    "Unit": "no2_unit"
}, inplace=True)

# 3. Filtrăm valorile valide
df_no2_filtered = df_no2[df_no2["no2"] > 1e-6].copy()
df_no2_filtered.reset_index(drop=True, inplace=True)

# 4. Adăugăm coordonatele stației B-4 (RO0065A)
df_no2_filtered["latitude"] = 44.4264
df_no2_filtered["longitude"] = 26.1406

# 5. Salvăm dataset-ul
df_no2_filtered.to_csv("data/no2-2022-e1a/no2_bucharest_clean-2022.csv", index=False)

print(f"✅ Cleaned dataset saved. Remaining rows: {len(df_no2_filtered)}")
df_no2_filtered.head()


✅ Cleaned dataset saved. Remaining rows: 8145


,start,end,no2,no2_unit,latitude,longitude
0,2022-01-01 00:00:00,2022-01-01 01:00:00,17.870000000000000000,ug.m-3,44.4264,26.1406
1,2022-01-01 01:00:00,2022-01-01 02:00:00,18.270000000000000000,ug.m-3,44.4264,26.1406
2,2022-01-01 02:00:00,2022-01-01 03:00:00,13.840000000000000000,ug.m-3,44.4264,26.1406
3,2022-01-01 03:00:00,2022-01-01 04:00:00,15.650000000000000000,ug.m-3,44.4264,26.1406
4,2022-01-01 04:00:00,2022-01-01 05:00:00,18.380000000000000000,ug.m-3,44.4264,26.1406


In [3]:
import pandas as pd

# 1. Încarcă fișierele
df_pm10 = pd.read_csv("../data/pm10-2022-e1a/pm10_bucharest_clean-2022.csv")
df_pm25 = pd.read_csv("../data/pm2.5-2022-e1a/pm25_bucharest_clean-2022.csv")
df_no2  = pd.read_csv("../data/no2-2022-e1a/no2_bucharest_clean-2022.csv")

# 2. Merge între PM10 și PM2.5 (doar dacă ambele au același interval)
df_merge_1 = pd.merge(df_pm10, df_pm25, on=["start", "end"], how="inner")

# 3. Merge între (PM10+PM2.5) și NO2
df_merged = pd.merge(df_merge_1, df_no2, on=["start", "end"], how="inner")

# 4. Păstrăm doar o singură coloană pentru lat și lon
df_merged["latitude"] = df_merged["latitude_x"]
df_merged["longitude"] = df_merged["longitude_x"]

# 5. Eliminăm coloanele duplicate
df_merged.drop(columns=[
    "pm10_unit", "pm2_5_unit", "no2_unit",
    "latitude_x", "longitude_x",
    "latitude_y", "longitude_y"
], inplace=True)

# 6. Reordonăm coloanele (opțional)
df_merged = df_merged[["start", "end", "pm10", "pm2_5", "no2", "latitude", "longitude"]]

# 7. Salvăm rezultatul
# df_merged.to_csv("data/bucharest_merged_pm10_pm25_no2.csv", index=False)

print(f"✅ Merged Dataset {len(df_merged)} rows.")
df_merged.head(10)



✅ Merged Dataset 7746 rows.


,start,end,pm10,pm2_5,no2,latitude,longitude
0,2022-01-02 00:00:00,2022-01-02 01:00:00,46.48,42.36,30.20,44.4264,26.1406
1,2022-01-02 01:00:00,2022-01-02 02:00:00,33.50,30.73,18.84,44.4264,26.1406
2,2022-01-02 02:00:00,2022-01-02 03:00:00,19.34,17.39,16.96,44.4264,26.1406
3,2022-01-02 03:00:00,2022-01-02 04:00:00,14.99,12.92,14.68,44.4264,26.1406
4,2022-01-02 04:00:00,2022-01-02 05:00:00,16.07,13.79,17.50,44.4264,26.1406
5,2022-01-02 05:00:00,2022-01-02 06:00:00,18.31,15.59,29.27,44.4264,26.1406
6,2022-01-02 06:00:00,2022-01-02 07:00:00,20.72,17.65,29.78,44.4264,26.1406
7,2022-01-02 07:00:00,2022-01-02 08:00:00,22.32,19.13,29.18,44.4264,26.1406
8,2022-01-02 08:00:00,2022-01-02 09:00:00,17.46,14.33,19.88,44.4264,26.1406
9,2022-01-02 09:00:00,2022-01-02 10:00:00,12.01,10.13,17.09,44.4264,26.1406


In [8]:
# 8. Încarcă datele meteo
df_weather = pd.read_csv("../data/meteo_data_2022/weather_bucharest_2022.csv")

# 9. Asigură-te că 'start' este datetime în ambele
df_merged["start"] = pd.to_datetime(df_merged["start"])
df_weather["start"] = pd.to_datetime(df_weather["start"])

# 10. Facem merge pe coloana 'start'
df_final = pd.merge(df_merged, df_weather, on="start", how="inner")

# Reorder columns
df_final = df_final[[
    "start", "end",
    "pm10", "pm2_5", "no2",
    "temperature", "humidity", "wind_speed", "pressure",
    "longitude", "latitude"
]]

# (Optional) Save again if needed
df_final.to_csv("data/bucharest_final_dataset_2022.csv", index=False)

print(f"✅ Final dataset combinat: {len(df_final)} rânduri.")
df_final.head(20)


✅ Final dataset combinat: 7745 rânduri.


,start,end,pm10,pm2_5,no2,temperature,humidity,wind_speed,pressure,longitude,latitude
0,2022-01-02 00:00:00,2022-01-02 01:00:00,46.48,42.36,30.20,4.8,100.0,7.2,1021.3,26.1406,44.4264
1,2022-01-02 01:00:00,2022-01-02 02:00:00,33.50,30.73,18.84,3.6,100.0,3.6,1021.8,26.1406,44.4264
2,2022-01-02 02:00:00,2022-01-02 03:00:00,19.34,17.39,16.96,2.9,100.0,3.6,1021.8,26.1406,44.4264
3,2022-01-02 03:00:00,2022-01-02 04:00:00,14.99,12.92,14.68,2.7,100.0,3.6,1022.1,26.1406,44.4264
4,2022-01-02 04:00:00,2022-01-02 05:00:00,16.07,13.79,17.50,2.2,100.0,3.6,1022.7,26.1406,44.4264
5,2022-01-02 05:00:00,2022-01-02 06:00:00,18.31,15.59,29.27,2.1,99.0,5.0,1023.4,26.1406,44.4264
6,2022-01-02 06:00:00,2022-01-02 07:00:00,20.72,17.65,29.78,1.6,100.0,3.6,1024.3,26.1406,44.4264
7,2022-01-02 07:00:00,2022-01-02 08:00:00,22.32,19.13,29.18,1.1,100.0,3.6,1025.4,26.1406,44.4264
8,2022-01-02 08:00:00,2022-01-02 09:00:00,17.46,14.33,19.88,2.6,100.0,3.6,1025.8,26.1406,44.4264
9,2022-01-02 09:00:00,2022-01-02 10:00:00,12.01,10.13,17.09,5.8,100.0,7.2,1026.0,26.1406,44.4264


In [12]:
print(df_final.loc[5000:5020])

                   start                  end   pm10  pm2_5    no2  \
5000 2022-08-27 17:00:00  2022-08-27 18:00:00  93.76  17.68   3.46   
5001 2022-08-27 18:00:00  2022-08-27 19:00:00  88.38  17.38   7.02   
5002 2022-08-27 19:00:00  2022-08-27 20:00:00  66.14  16.05  28.77   
5003 2022-08-27 21:00:00  2022-08-27 22:00:00  71.61  24.50  60.19   
5004 2022-08-27 22:00:00  2022-08-27 23:00:00  74.43  26.79  68.00   
5005 2022-08-27 23:00:00  2022-08-28 00:00:00  83.19  25.02  66.27   
5006 2022-08-28 00:00:00  2022-08-28 01:00:00  77.76  25.29  40.17   
5007 2022-08-28 01:00:00  2022-08-28 02:00:00  67.28  24.63  34.34   
5008 2022-08-28 02:00:00  2022-08-28 03:00:00  70.14  22.18  30.59   
5009 2022-08-28 03:00:00  2022-08-28 04:00:00  66.03  19.35  23.01   
5010 2022-08-28 04:00:00  2022-08-28 05:00:00  63.10  19.57  20.03   
5011 2022-08-28 05:00:00  2022-08-28 06:00:00  57.25  18.94  17.53   
5012 2022-08-28 06:00:00  2022-08-28 07:00:00  56.52  19.75  17.84   
5013 2022-08-28 07:0